# Preprocessing

We retrieve our data from https://asrs.arc.nasa.gov/search/database.html (NASA’s Aviation Safety Reporting System) to analyze pilot and controller narratives to gain more insight into midair collisions and the factors affecting them. Our study are limited to the narratives on the ASRS website to collisions. We do not differentiate between near midair collisions and actual midair collisions, since they are both events we wish to understand better.

In [14]:
import numpy as np
import json as js
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
# Retrieve the data from the csv file
data = pd.read_csv("02-ASRS_DBOnline.csv",index_col=0).reset_index()

# Dropping the entires that do not have relative distance
data = data.dropna(subset=['Place.3','Place.5','Report 1.2']).reset_index()
data.head()

,index,,Time,Time.1,Place,Place.1,Place.2,Place.3,Place.4,Place.5,...,Events.4,Events.5,Assessments,Assessments.1,Report 1,Report 1.1,Report 2,Report 2.1,Report 1.2,Unnamed: 96
0,0,ACN,Date,Local Time Of Day,Locale Reference,State Reference,Relative Position.Angle.Radial,Relative Position.Distance.Nautical Miles,Altitude.AGL.Single Value,Altitude.MSL.Single Value,...,When Detected,Result,Contributing Factors / Situations,Primary Problem,Narrative,Callback,Narrative,Callback,Synopsis,NaN
1,1,81135,198801,0601-1200,BDR; 0N0,CT,360,9,NaN,2500,...,NaN,Flight Crew Took Evasive Action,NaN,Human Factors,F/O FLYING ACFT FROM LEFT SEAT. BLUE AND WHITE...,NaN,NaN,NaN,HELICOPTER ENCOUNTERED SMA ACFT IN SEE AND AVO...,NaN
2,2,81311,198801,1201-1800,BAF,MA,NaN,22,NaN,4500,...,NaN,Flight Crew Took Evasive Action,NaN,Human Factors,WHILE LEVEL AT 4500'; HDG 246 DEGS ON VICTOR 1...,NaN,NaN,NaN,CLOSE PROX GA-SMA GA-SMT ON V106.,NaN
3,3,81663,198801,0601-1200,PYM,MA,NaN,2,NaN,800,...,NaN,Flight Crew Executed Go Around / Missed Approach,NaN,Human Factors,I WAS INVOLVED IN A NMAC WHILE ON FINAL APCH T...,NaN,NaN,NaN,CLOSE PROX 2 GA-SMA ACFT IN TRAFFIC PATTERN AT...,NaN
4,4,81862,198802,1201-1800,BED,MA,NaN,15,NaN,3500,...,NaN,Flight Crew Took Evasive Action,NaN,Human Factors,WHILE ON CROSS-COUNTRY VFR FROM ORANGE; MA TO ...,NaN,NaN,NaN,CLOSE PROX GA SMA AND PARACHUTIST IN RESTRICTE...,NaN


In [16]:
# Extracting reports for each entry
report1 = data['Report 1']
report2 = data['Report 1.2']

In [17]:
# Preprocessing of the data by stemming 
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

ps = PorterStemmer()
new_report = []
word_data = []

insignificant_terms = ['accident','us','were','our','we','NMAC','air','data','follow','in','the','had','for','from','on','to','with','and','while','','than','less','in','of','at','an','has']
for i in range(0,len(report2)):
    temp_report = ''
    # Stem the narratives of each report
    if pd.isnull(report2[i]) != True:
        words = word_tokenize(report2[i])
        for w in words:
            if w not in word_data:
                word_data.append([w, ps.stem(w)])
            w = ps.stem(w)
            # Add the insignificant words here 
            if w not in insignificant_terms:
                temp_report += (w + " ")
        words = word_tokenize(report1[i])
        for w in words:
            if w not in word_data:
                word_data.append([w, ps.stem(w)])
            w = ps.stem(w)
            # Add the insignificant words here 
            if w not in insignificant_terms:
                temp_report += (w + " ")
        new_report.append((temp_report))

[nltk_data] Downloading package punkt to /Users/m804/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
report_stemmed = pd.DataFrame({'Report': new_report, 'Place.3': data['Place.3'], 'Place.5': data['Place.5']})
report_stemmed = report_stemmed[1:]
report_stemmed.to_csv('02-ASRS-stemmed.csv')
report_stemmed.head()

,Place.3,Place.5,Report
1,9,2500,helicopt encount sma acft IN see avoid situat ...
2,22,4500,close prox ga-sma ga-smt ON v106 . level AT 45...
3,2,800,close prox 2 ga-sma acft IN traffic pattern AT...
4,15,3500,close prox GA sma parachutist IN restrict area...
5,2,1800,standard separaton nmac between commut sma acf...


In [19]:
# Save word reference
word_reference = pd.DataFrame(word_data, columns=["before", "after"])
print(word_reference.shape)
word_reference.to_csv('02-ASRS-word-reference.csv')
word_reference.head()

(153747, 2)


,before,after
0,Synopsis,synopsi
1,Narrative,narr
2,HELICOPTER,helicopt
3,ENCOUNTERED,encount
4,SMA,sma


In [20]:
# read the scrapped dataset
report = pd.read_csv("01-NTSB-stemmed.csv", index_col=0)

In [21]:
# Concat two datasets
new_report = (new_report) + report['Summary'].tolist()
df_new_report = pd.DataFrame(new_report, columns=["Summary"])
df_new_report.to_csv('02-report.csv')
df_new_report.head()

,Summary
0,synopsi narr
1,helicopt encount sma acft IN see avoid situat ...
2,close prox ga-sma ga-smt ON v106 . level AT 45...
3,close prox 2 ga-sma acft IN traffic pattern AT...
4,close prox GA sma parachutist IN restrict area...
